In [1]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.datasets import imdb 
from sklearn.metrics import classification_report

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
lm = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics, svm

In [5]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
n_unique_words = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=n_unique_words)

In [8]:
maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [104]:
x_train[0].shape

(200,)

In [6]:
#pure bilstm
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [7]:
#model.fit(x_train, y_train, validation_split = 0.2,epochs = 12, verbose = True,batch_size = 64)

In [22]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [38]:
inp = Input(shape=(200, ))
x = Embedding(10000, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_1)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 200)]             0         
                                                                 
 embedding_12 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 12, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 bidirectional_10 (Bidirecti  (None, 12, 128)          49664     
 onal)                                                           
                                                           

In [39]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es

In [40]:
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")

In [41]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#file_path = 'model.hdf5'
#ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#early = EarlyStopping(monitor="val_loss", mode="min", patience=1)

In [42]:
model.fit(x_train, y_train, batch_size=256, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
79/79 [==============================] - 19s 179ms/step - loss: 0.6889 - accuracy: 0.5337 - val_loss: 0.6628 - val_accuracy: 0.5972
Epoch 2/20
79/79 [==============================] - 14s 173ms/step - loss: 0.6039 - accuracy: 0.6755 - val_loss: 0.5384 - val_accuracy: 0.7326
Epoch 3/20
79/79 [==============================] - 14s 174ms/step - loss: 0.5147 - accuracy: 0.7505 - val_loss: 0.5073 - val_accuracy: 0.7514
Epoch 4/20
79/79 [==============================] - 14s 176ms/step - loss: 0.4641 - accuracy: 0.7862 - val_loss: 0.4748 - val_accuracy: 0.7680
Epoch 5/20
79/79 [==============================] - 14s 181ms/step - loss: 0.4103 - accuracy: 0.8184 - val_loss: 0.4901 - val_accuracy: 0.7654
Epoch 6/20
79/79 [==============================] - 14s 183ms/step - loss: 0.3811 - accuracy: 0.8316 - val_loss: 0.4464 - val_accuracy: 0.7908
Epoch 7/20
79/79 [==============================] - 15s 184ms/step - loss: 0.3389 - accuracy: 0.8550 - val_loss: 0.4554 - val_accuracy: 0.7880

In [43]:
y_pred = model.predict(x_test)

782/782 [==============================] - 7s 9ms/step


In [44]:
y_pred.shape

(25000, 1)

In [45]:
y_test.shape

(25000,)

In [46]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [47]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77     12500
           1       0.75      0.87      0.80     12500

    accuracy                           0.79     25000
   macro avg       0.80      0.79      0.79     25000
weighted avg       0.80      0.79      0.79     25000



In [48]:
new_df = pd.read_csv('Data_set.txt',delimiter = "\t",encoding='latin-1')

In [49]:
new_df.head()

,Rating,Description
0,0,what's surprising about this traditional thril...
1,0,the essential problem in orange county is that...
2,1,a devastating indictment of unbridled greed an...
3,1,a breezy romantic comedy that has the punch of...
4,1,drumline ably captures the complicated relatio...


In [50]:
y = new_df['Rating']
X = new_df['Description']
def clean_data(data):
    data = data.replace("\n", "")
    data = re.sub(r'\W', ' ', data)
    data = re.sub(r'\s+[a-zA-Z]\s+', ' ', data)
    data = re.sub(r'\s+', ' ', data, flags=re.I)
    data = data.lower().split(" ")
    data = [word for word in data if word not in set(stopwords.words("english"))]                                            
    data = ' '.join([str(elem) for elem in data])
    return data

In [51]:
incrementer=0
for i in X:
    i = clean_data(i)
    X[incrementer]=i
    incrementer+=1

C:\Users\HP\AppData\Local\Temp\ipykernel_20368\2672807160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[incrementer]=i


In [52]:
incrementer=0
for i in X:
    listw = nltk.word_tokenize(i)
    lemmatized_string = ' '.join([lm.lemmatize(words) for words in listw])
    X[incrementer]=lemmatized_string
    incrementer+=1

C:\Users\HP\AppData\Local\Temp\ipykernel_20368\3635973817.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[incrementer]=lemmatized_string


In [105]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

In [115]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
print(X_train[0])
vocab_size = len(tokenizer.word_index) + 1 
print(vocab_size)

[745, 291, 88, 746, 122, 549, 350, 69, 519, 3, 216, 401]
14436


In [107]:
maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

X_train[0].shape

(200,)

In [108]:
vocab_size

14436

In [109]:
inp = Input(shape=(200, ))
x = Embedding(14436, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_1)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 200)]             0         
                                                                 
 embedding_15 (Embedding)    (None, 200, 128)          1847808   
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 12, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 bidirectional_13 (Bidirecti  (None, 12, 128)          49664     
 onal)                                                           
                                                           

In [110]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
100/100 [==============================] - 10s 51ms/step - loss: 0.6903 - accuracy: 0.5439 - val_loss: 0.6881 - val_accuracy: 0.5506
Epoch 2/20
100/100 [==============================] - 5s 47ms/step - loss: 0.6903 - accuracy: 0.5457 - val_loss: 0.6890 - val_accuracy: 0.5506
Epoch 3/20
100/100 [==============================] - 5s 50ms/step - loss: 0.6892 - accuracy: 0.5444 - val_loss: 0.6890 - val_accuracy: 0.5506
Epoch 4/20
100/100 [==============================] - 5s 51ms/step - loss: 0.6723 - accuracy: 0.5813 - val_loss: 0.6868 - val_accuracy: 0.5638
Epoch 5/20
100/100 [==============================] - 5s 51ms/step - loss: 0.6229 - accuracy: 0.6534 - val_loss: 0.7013 - val_accuracy: 0.5581
Epoch 6/20
100/100 [==============================] - 5s 51ms/step - loss: 0.5626 - accuracy: 0.7042 - val_loss: 0.7231 - val_accuracy: 0.5631
Epoch 7/20
100/100 [==============================] - 5s 51ms/step - loss: 0.5051 - accuracy: 0.7378 - val_loss: 0.7885 - val_accuracy: 0.568

In [111]:
y_pred = model.predict(X_test)

84/84 [==============================] - 1s 6ms/step


In [112]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [113]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.60      0.75      0.67      1492
           1       0.54      0.38      0.45      1174

    accuracy                           0.58      2666
   macro avg       0.57      0.56      0.56      2666
weighted avg       0.58      0.58      0.57      2666



In [119]:
df_imdb = pd.read_csv('C:/Users/HP/IMDB Dataset/IMDB Dataset.csv',)

In [120]:
df_imdb['review'] = df_imdb['review'].str.lower()

In [124]:
import string

In [123]:
def remove_stopwords(df_imdb):
    df_imdb['review without stopwords'] = df_imdb['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords.words("english"))]))
    return df_imdb

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(df_imdb)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


NameError: name 'string' is not defined

In [125]:
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

C:\Users\HP\AppData\Local\Temp\ipykernel_20368\706899345.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [126]:
data_without_stopwords.head()


,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching 1 oz episode ...,one reviewers mentioned watching 1 oz episode ...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically there's family little boy (jake) thi...,basically there s family little boy jake thi...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [127]:
data_without_stopwords.to_csv('IMDB Dataset/cleaned_data.csv')
reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching 1 oz episode ...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically there s family little boy  jake  thi...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  creative origina...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    i m going disagree previous comment side malti...
49999    one expects star trek movies high art  fans ex...
Name: clean_review, Length: 50000, dtype: object

In [128]:
reviews_list = []
for i in range(len(reviews)):
    reviews_list.append(reviews[i])

In [129]:
sentiment = data_without_stopwords['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

In [130]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [152]:
X_train[0]

'saw film birmingham southern college 1975  shown combination red balloon  films similar dream like quality  bulk film entails fish swimming happily bowl new owner  little boy  away school  cat enters room fish bowl are  begins warily stalk  prey   boy begins walk home school  viewer wonders whether arrive time save fish friend  fish becomes agitated cat s presence  finally jumps bowl  cat quickly walks fish  gently picks paws  returns bowl  boy returns happily fish  none wiser the ending amazing irony technical complexity  hard imagine director could ve pulled technical feat back 1959    seems trick 2003 if find it  watch    disappointed   do  find it  let know get copy  too '

In [131]:
len(Y_train)

40000

In [157]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
     

words_to_index = tokenizer.word_index
     

len(words_to_index)

95417

In [158]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map
    
     

In [159]:
word_to_vec_map = read_glove_vector('C:/Users/HP/Downloads/glove.6B/glove.6B.50d.txt')

In [160]:
maxLen = 150
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector






In [161]:
inp = Input(shape=(150, ))
x = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)(inp)
conv_1 = Conv1D(512, 3, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(3)(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
x = Bidirectional(LSTM(32, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_1)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 150)]             0         
                                                                 
 embedding_27 (Embedding)    (None, 150, 50)           4770850   
                                                                 
 conv1d_1 (Conv1D)           (None, 148, 512)          77312     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 49, 512)          0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 49, 512)           0         
                                                                 
 bidirectional_15 (Bidirecti  (None, 49, 64)           139520    
 onal)                                                     

In [162]:
X_train_indices = tokenizer.texts_to_sequences(X_train)
     

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [163]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
     

In [165]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_indices, Y_train, batch_size=64, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
500/500 [==============================] - 139s 271ms/step - loss: 0.4892 - accuracy: 0.7623 - val_loss: 0.3939 - val_accuracy: 0.8211
Epoch 2/20
500/500 [==============================] - 134s 268ms/step - loss: 0.3747 - accuracy: 0.8369 - val_loss: 0.3747 - val_accuracy: 0.8363
Epoch 3/20
500/500 [==============================] - 135s 271ms/step - loss: 0.3349 - accuracy: 0.8562 - val_loss: 0.3457 - val_accuracy: 0.8485
Epoch 4/20
500/500 [==============================] - 134s 269ms/step - loss: 0.3089 - accuracy: 0.8695 - val_loss: 0.3454 - val_accuracy: 0.8400
Epoch 5/20
500/500 [==============================] - 136s 271ms/step - loss: 0.2855 - accuracy: 0.8798 - val_loss: 0.3283 - val_accuracy: 0.8609
Epoch 6/20
500/500 [==============================] - 135s 271ms/step - loss: 0.2612 - accuracy: 0.8928 - val_loss: 0.3295 - val_accuracy: 0.8586
Epoch 7/20
500/500 [==============================] - 136s 271ms/step - loss: 0.2440 - accuracy: 0.8986 - val_loss: 0.3987 -

In [166]:
y_pred = model.predict(X_test_indices)

313/313 [==============================] - 7s 20ms/step


In [167]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [169]:
print(classification_report(Y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.94      0.74      0.83      5035
           1       0.78      0.95      0.86      4965

    accuracy                           0.84     10000
   macro avg       0.86      0.84      0.84     10000
weighted avg       0.86      0.84      0.84     10000

